#import the csv into pandas df
#clean the table name and ensure item format
#clean the column headers 
#write the create table SQL statement (DONE already)
#import the data into the db

In [70]:
import os
import numpy as np
import pandas as pd
import psycopg2
import re

In [71]:
file_path = "../reports_from_branches/chesterfield_25-08-2021_09-00-00.csv"
column_names = ["date_time", "branch", "customer", "order_content", "total_price", "payment_type", "credit_card_number"]
df = pd.read_csv(file_path, names=column_names)
df.head()

,date_time,branch,customer,order_content,total_price,payment_type,credit_card_number
0,25/08/2021 09:00,Chesterfield,Richard Copeland,Regular Flavoured iced latte - Hazelnut - 2.75...,5.2,CARD,5.494174e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,"Large Flavoured iced latte - Caramel - 3.25, R...",17.3,CARD,6.844802e+15
2,25/08/2021 09:04,Chesterfield,Francis Strayhorn,"Large Flat white - 2.45, Regular Latte - 2.15",4.6,CARD,9.557104e+15
3,25/08/2021 09:06,Chesterfield,Rodney Drake,"Regular Flavoured latte - Hazelnut - 2.55, Lar...",5.0,CARD,2.978328e+15
4,25/08/2021 09:08,Chesterfield,Michael Sparrow,"Regular Latte - 2.15, Large Latte - 2.45",4.6,CASH,NaN


In [72]:
#spliting the first time, turn order_content into a list
df["order_content"]= df["order_content"].str.split(',')
df.head()

,date_time,branch,customer,order_content,total_price,payment_type,credit_card_number
0,25/08/2021 09:00,Chesterfield,Richard Copeland,[Regular Flavoured iced latte - Hazelnut - 2.7...,5.2,CARD,5.494174e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,"[Large Flavoured iced latte - Caramel - 3.25, ...",17.3,CARD,6.844802e+15
2,25/08/2021 09:04,Chesterfield,Francis Strayhorn,"[Large Flat white - 2.45, Regular Latte - 2.15]",4.6,CARD,9.557104e+15
3,25/08/2021 09:06,Chesterfield,Rodney Drake,"[Regular Flavoured latte - Hazelnut - 2.55, L...",5.0,CARD,2.978328e+15
4,25/08/2021 09:08,Chesterfield,Michael Sparrow,"[Regular Latte - 2.15, Large Latte - 2.45]",4.6,CASH,NaN


In [73]:
print(df["order_content"])


0      [Regular Flavoured iced latte - Hazelnut - 2.7...
1      [Large Flavoured iced latte - Caramel - 3.25, ...
2       [Large Flat white - 2.45,  Regular Latte - 2.15]
3      [Regular Flavoured latte - Hazelnut - 2.55,  L...
4            [Regular Latte - 2.15,  Large Latte - 2.45]
                             ...                        
263    [Large Flavoured iced latte - Vanilla - 3.25, ...
264    [Large Flavoured latte - Hazelnut - 2.85,  Lar...
265    [Large Flavoured iced latte - Hazelnut - 3.25,...
266          [Regular Flavoured latte - Hazelnut - 2.55]
267                            [Large Flat white - 2.45]
Name: order_content, Length: 268, dtype: object


In [74]:
#spliting the order_content into individual row
df.explode("order_content")

,date_time,branch,customer,order_content,total_price,payment_type,credit_card_number
0,25/08/2021 09:00,Chesterfield,Richard Copeland,Regular Flavoured iced latte - Hazelnut - 2.75,5.20,CARD,5.494174e+15
0,25/08/2021 09:00,Chesterfield,Richard Copeland,Large Latte - 2.45,5.20,CARD,5.494174e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,Large Flavoured iced latte - Caramel - 3.25,17.30,CARD,6.844802e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,Regular Flavoured iced latte - Hazelnut - 2.75,17.30,CARD,6.844802e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,Regular Flavoured iced latte - Caramel - 2.75,17.30,CARD,6.844802e+15
...,...,...,...,...,...,...,...
265,25/08/2021 16:52,Chesterfield,Sharon Campanelli,Regular Flavoured iced latte - Caramel - 2.75,10.70,CASH,NaN
265,25/08/2021 16:52,Chesterfield,Sharon Campanelli,Regular Flavoured latte - Hazelnut - 2.55,10.70,CASH,NaN
265,25/08/2021 16:52,Chesterfield,Sharon Campanelli,Regular Flat white - 2.15,10.70,CASH,NaN
266,25/08/2021 16:56,Chesterfield,Henry Frew,Regular Flavoured latte - Hazelnut - 2.55,2.55,CASH,NaN


In [75]:
df.head()

,date_time,branch,customer,order_content,total_price,payment_type,credit_card_number
0,25/08/2021 09:00,Chesterfield,Richard Copeland,[Regular Flavoured iced latte - Hazelnut - 2.7...,5.2,CARD,5.494174e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,"[Large Flavoured iced latte - Caramel - 3.25, ...",17.3,CARD,6.844802e+15
2,25/08/2021 09:04,Chesterfield,Francis Strayhorn,"[Large Flat white - 2.45, Regular Latte - 2.15]",4.6,CARD,9.557104e+15
3,25/08/2021 09:06,Chesterfield,Rodney Drake,"[Regular Flavoured latte - Hazelnut - 2.55, L...",5.0,CARD,2.978328e+15
4,25/08/2021 09:08,Chesterfield,Michael Sparrow,"[Regular Latte - 2.15, Large Latte - 2.45]",4.6,CASH,NaN


In [76]:
df.dtypes

date_time              object
branch                 object
customer               object
order_content          object
total_price           float64
payment_type           object
credit_card_number    float64
dtype: object

In [77]:
replacements = {
    'object' : 'varchar',
    'float64' : 'float'
}



In [78]:
#switching and joining datatype from pandas to sql
col_str = ", ".join("{} {}". format(n, d) for (n, d)in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'date_time varchar, branch varchar, customer varchar, order_content varchar, total_price float, payment_type varchar, credit_card_number float'

In [79]:
#wrting sql command
creat_table_sql = "create table raw_data" + f"({col_str});"
creat_table_sql

'create table raw_data(date_time varchar, branch varchar, customer varchar, order_content varchar, total_price float, payment_type varchar, credit_card_number float);'

In [80]:
#establish connection
conn = psycopg2.connect(
    database="postgres",
    user = "postgres",
    password = "pass",
    host = "127.0.0.1",
    port = "5432"
)
cursor = conn.cursor()
print('opened database successfully')

opened database successfully


In [81]:
#drop table
cursor.execute("drop table if exists raw_data;")

In [82]:
#create table
cursor.execute(creat_table_sql)

In [83]:
#insert values to table

#save df to csv
df.to_csv(file_path, header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object, and upload to db
my_file = open(file_path)
print("file opened in memory")

file opened in memory


In [84]:
#upload to db
sql_statement = """
COPY raw_data FROM STDIN WITH
CSV
HEADER
DELIMITER AS ','
"""

cursor.copy_expert(sql=sql_statement, file=my_file)
print('file copied to db')


file copied to db
